In [51]:
import pandas as pd
df = pd.read_csv("../data/preprocessed.csv")

In [52]:
df[:100]

,time,event_type,channel,value
0,2023-10-11 00:00:32.787951,e,1,6
1,2023-10-11 00:00:32.787951,e,2,12
2,2023-10-11 00:00:32.787951,e,3,14
3,2023-10-11 00:00:32.787951,e,4,16
4,2023-10-11 00:00:32.787951,e,5,15
...,...,...,...,...
95,2023-10-11 00:10:32.803796,e,6,27
96,2023-10-11 00:10:32.803796,e,7,93
97,2023-10-11 00:10:32.803796,e,8,872
98,2023-10-11 00:10:32.803796,e,9,0


In [124]:
import influxdb_client, os, time
from influxdb_client import InfluxDBClient, Point, WritePrecision
from influxdb_client.client.write_api import SYNCHRONOUS

token = os.environ.get("INFLUXDB_TOKEN")
org = "radem"
url = "http://localhost:8086"

client = influxdb_client.InfluxDBClient(url=url, token=token, org=org)

write_api = client.write_api(write_options=SYNCHRONOUS)
query_api = client.query_api()

for record in df[:1000].iterrows():
    point = Point("df2") \
        .time(pd.to_datetime(record[1]["time"])) \
        .tag("event_type", record[1]["event_type"]) \
        .tag("channel", record[1]["channel"]) \
        .field("value", record[1]["value"])
    write_api.write(bucket="data", record=point)


write_api.close()

In [125]:
## make query to database to get data
query = f'from(bucket:"data") |> range(start: 0) |> filter(fn: (r) => r._measurement == "df")'
tables = query_api.query(query, org=org)

for table in tables:
    for record in table.records:
        print(record.values)

{'result': '_result', 'table': 0, '_start': datetime.datetime(1970, 1, 1, 0, 0, tzinfo=tzutc()), '_stop': datetime.datetime(2024, 3, 21, 15, 3, 5, 110296, tzinfo=tzutc()), '_time': datetime.datetime(2023, 10, 11, 0, 0, 32, 787951, tzinfo=tzutc()), '_value': 6, '_field': 'value', '_measurement': 'df', 'channel': '1', 'event_type': 'e'}
{'result': '_result', 'table': 0, '_start': datetime.datetime(1970, 1, 1, 0, 0, tzinfo=tzutc()), '_stop': datetime.datetime(2024, 3, 21, 15, 3, 5, 110296, tzinfo=tzutc()), '_time': datetime.datetime(2023, 10, 11, 0, 1, 32, 789510, tzinfo=tzutc()), '_value': 1, '_field': 'value', '_measurement': 'df', 'channel': '1', 'event_type': 'e'}
{'result': '_result', 'table': 0, '_start': datetime.datetime(1970, 1, 1, 0, 0, tzinfo=tzutc()), '_stop': datetime.datetime(2024, 3, 21, 15, 3, 5, 110296, tzinfo=tzutc()), '_time': datetime.datetime(2023, 10, 11, 0, 2, 32, 791175, tzinfo=tzutc()), '_value': 10, '_field': 'value', '_measurement': 'df', 'channel': '1', 'event_